Based on business goals, I will focus on engineering features that directly contribute to understanding EV adoption trends, consumer behavior, and regional preferences.

In [1]:
#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#to display plots inline
%matplotlib inline

In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Access your dataset
data_path = '/content/gdrive/My Drive/Colab Notebooks/Electiric vehical/final_after_outlier_cleaned_data.csv'

In [4]:
df = pd.read_csv(data_path)
# Display the first few rows of the DataFrame
df.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,1C4JJXP68P,Yakima,Yakima,WA,98901.0,2023,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,21,0,249905295,POINT (-120.4688751 46.6046178),PACIFICORP,5.307700e+10
1,WBY8P6C05L,Kitsap,Kingston,WA,98346.0,2020,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,153,0,260917289,POINT (-122.5178351 47.7981436),PUGET SOUND ENERGY INC,5.303509e+10
2,JTDKARFP1J,Kitsap,Port Orchard,WA,98367.0,2018,TOYOTA,PRIUS PRIME,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,25,0,186410087,POINT (-122.6530052 47.4739066),PUGET SOUND ENERGY INC,5.303509e+10
3,5UXTA6C09N,Snohomish,Everett,WA,98208.0,2022,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,30,0,186076915,POINT (-122.2032349 47.8956271),PUGET SOUND ENERGY INC,5.306104e+10
4,JTMAB3FV7P,Thurston,Rainier,WA,98576.0,2023,TOYOTA,RAV4 PRIME,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,42,0,236505139,POINT (-122.6771414 46.8882415),PUGET SOUND ENERGY INC,5.306701e+10


In [7]:
print(df.columns)


Index(['VIN (1-10)', 'County', 'City', 'State', 'Postal Code', 'Model Year',
       'Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'DOL Vehicle ID', 'Vehicle Location', 'Electric Utility',
       '2020 Census Tract', 'Make_Proportion'],
      dtype='object')


**Review the Existing Features:**

Before creating new features, it’s essential to review the existing ones:

- Urban vs. Non-Urban Classification
- CAFV Eligibility Category (High vs. Low)
- Make (Manufacturer)
- Model Year
- County/City
- Electric Vehicle Type (BEV vs. PHEV)

# Feature Engineering for Geographic Distribution Analysis

**Regional Popularity of Makes:**

Purpose: Understand if certain manufacturers are more popular in specific regions.

Method: Calculate the proportion of vehicles from each manufacturer (e.g., Tesla, Nissan) in each county.

In [5]:
df['Make_Proportion'] = df.groupby('County')['Make'].transform(lambda x: x.value_counts(normalize=True))


**Interaction Terms: Urbanization and CAFV Eligibility:**

Purpose: Capture the combined effect of urbanization and CAFV eligibility on EV adoption.

Method: Create interaction terms between Urban and CAFV Eligibility Category.

**Creating an 'Urban' Column Based on Available Data**

In [8]:
# List of urban cities (this is just an example, replace with actual cities)
urban_cities = ['Seattle', 'Bellevue', 'Redmond', 'Tacoma', 'Vancouver']

# Create the 'Urban' column based on the City column
df['Urban'] = df['City'].apply(lambda x: 'Urban' if x in urban_cities else 'Non-Urban')

# Alternatively, create the 'Urban' column based on the County column (if more appropriate)
urban_counties = ['King', 'Pierce', 'Snohomish', 'Clark', 'Thurston']
df['Urban'] = df['County'].apply(lambda x: 'Urban' if x in urban_counties else 'Non-Urban')

# Now, i can proceed with the feature engineering and modeling steps that require the 'Urban' column.


In [13]:
print(df.columns)

Index(['VIN (1-10)', 'County', 'City', 'State', 'Postal Code', 'Model Year',
       'Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'DOL Vehicle ID', 'Vehicle Location', 'Electric Utility',
       '2020 Census Tract', 'Make_Proportion', 'Urban', 'CAFV Eligibility'],
      dtype='object')


In [16]:
# If you want to combine the 'Urban' and 'CAFV Eligibility' columns:
df['Urban_CAFV_Interaction'] = df['Urban'].astype(str) + '_' + df['CAFV Eligibility'].astype(str)

# Then, apply one-hot encoding to the new interaction column:
df = pd.get_dummies(df, columns=['Urban_CAFV_Interaction'], drop_first=True)



**Vehicle Age**

Purpose: Understand how the age of vehicles impacts their adoption in different regions.

Method: Create a new feature that calculates the vehicle's age by subtracting the Model Year from the current year.

In [18]:
import datetime
current_year = datetime.datetime.now().year
df['Vehicle Age'] = current_year - df['Model Year']


In [19]:
df.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,...,Base MSRP,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract,Make_Proportion,Urban,CAFV Eligibility,Urban_CAFV_Interaction_Urban_Not Eligible,Vehicle Age
0,1C4JJXP68P,Yakima,Yakima,WA,98901.0,2023,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,...,0,249905295,POINT (-120.4688751 46.6046178),PACIFICORP,5.307700e+10,NaN,Non-Urban,Not Eligible,False,1
1,WBY8P6C05L,Kitsap,Kingston,WA,98346.0,2020,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,...,0,260917289,POINT (-122.5178351 47.7981436),PUGET SOUND ENERGY INC,5.303509e+10,NaN,Non-Urban,Not Eligible,False,4
2,JTDKARFP1J,Kitsap,Port Orchard,WA,98367.0,2018,TOYOTA,PRIUS PRIME,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,...,0,186410087,POINT (-122.6530052 47.4739066),PUGET SOUND ENERGY INC,5.303509e+10,NaN,Non-Urban,Not Eligible,False,6
3,5UXTA6C09N,Snohomish,Everett,WA,98208.0,2022,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,...,0,186076915,POINT (-122.2032349 47.8956271),PUGET SOUND ENERGY INC,5.306104e+10,NaN,Urban,Not Eligible,True,2
4,JTMAB3FV7P,Thurston,Rainier,WA,98576.0,2023,TOYOTA,RAV4 PRIME,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,...,0,236505139,POINT (-122.6771414 46.8882415),PUGET SOUND ENERGY INC,5.306701e+10,NaN,Urban,Not Eligible,True,1


In [20]:
print(df.columns)

Index(['VIN (1-10)', 'County', 'City', 'State', 'Postal Code', 'Model Year',
       'Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'DOL Vehicle ID', 'Vehicle Location', 'Electric Utility',
       '2020 Census Tract', 'Make_Proportion', 'Urban', 'CAFV Eligibility',
       'Urban_CAFV_Interaction_Urban_Not Eligible', 'Vehicle Age'],
      dtype='object')


# Feature Engineering for Market Penetration and Growth Trends

**Yearly Growth Rates**

Purpose: Understand how EV adoption grows over time within each region.

Method: Calculate the year-over-year growth rate of EV registrations.

In [22]:
# Step 1: Group by County and Model Year and count the number of Electric Vehicle Types
yearly_ev_count = df.groupby(['County', 'Model Year'])['Electric Vehicle Type'].count().reset_index(name='EV_Count')

# Step 2: Calculate the percentage change (yearly growth rate) within each County
yearly_ev_count['Yearly_Growth_Rate'] = yearly_ev_count.groupby('County')['EV_Count'].pct_change()

# Step 3: Merge the yearly growth rate back to the original dataframe (if needed)
df = df.merge(yearly_ev_count[['County', 'Model Year', 'Yearly_Growth_Rate']], on=['County', 'Model Year'], how='left')


In [23]:
df['Yearly_Growth_Rate'] = df['Yearly_Growth_Rate'].fillna(0)  # Optionally fill NaNs with 0 or another appropriate value


**Time Series Features**

Purpose: Capture trends over time for better predictive modeling.

Method: Create features such as Year, Quarter, and Month to capture seasonality and trends in EV adoption.

In [26]:
df['Quarter'] = pd.to_datetime(df['Model Year'], format='%Y').dt.quarter
df['Month'] = pd.to_datetime(df['Model Year'], format='%Y').dt.month


In [28]:
print(df.columns)

Index(['VIN (1-10)', 'County', 'City', 'State', 'Postal Code', 'Model Year',
       'Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'DOL Vehicle ID', 'Vehicle Location', 'Electric Utility',
       '2020 Census Tract', 'Make_Proportion', 'Urban', 'CAFV Eligibility',
       'Urban_CAFV_Interaction_Urban_Not Eligible', 'Vehicle Age',
       'Yearly_Growth_Rate', 'Quarter', 'Month'],
      dtype='object')


In [29]:
df.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,...,Electric Utility,2020 Census Tract,Make_Proportion,Urban,CAFV Eligibility,Urban_CAFV_Interaction_Urban_Not Eligible,Vehicle Age,Yearly_Growth_Rate,Quarter,Month
0,1C4JJXP68P,Yakima,Yakima,WA,98901.0,2023,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,...,PACIFICORP,5.307700e+10,NaN,Non-Urban,Not Eligible,False,1,0.454082,1,1
1,WBY8P6C05L,Kitsap,Kingston,WA,98346.0,2020,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,...,PUGET SOUND ENERGY INC,5.303509e+10,NaN,Non-Urban,Not Eligible,False,4,-0.506024,1,1
2,JTDKARFP1J,Kitsap,Port Orchard,WA,98367.0,2018,TOYOTA,PRIUS PRIME,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,...,PUGET SOUND ENERGY INC,5.303509e+10,NaN,Non-Urban,Not Eligible,False,6,0.149007,1,1
3,5UXTA6C09N,Snohomish,Everett,WA,98208.0,2022,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,...,PUGET SOUND ENERGY INC,5.306104e+10,NaN,Urban,Not Eligible,True,2,0.485420,1,1
4,JTMAB3FV7P,Thurston,Rainier,WA,98576.0,2023,TOYOTA,RAV4 PRIME,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,...,PUGET SOUND ENERGY INC,5.306701e+10,NaN,Urban,Not Eligible,True,1,1.195812,1,1


# Feature Engineering for Consumer Behavior Analysis

**Preference Ratio for BEVs vs. PHEVs**

Purpose: Understand regional preferences for BEVs vs. PHEVs.

Method: Calculate the proportion of BEVs and PHEVs within each county or city.

In [30]:
df['BEV_Proportion'] = df.groupby('County')['Electric Vehicle Type'].transform(lambda x: (x == 'BEV').mean())
df['PHEV_Proportion'] = df.groupby('County')['Electric Vehicle Type'].transform(lambda x: (x == 'PHEV').mean())


**Manufacturer Dominance**

Purpose: Identify if specific manufacturers dominate certain regions, which could indicate local preferences or incentives.

Method: Calculate the proportion of each manufacturer within each region and create a feature to represent the dominant manufacturer.

In [31]:
df['Dominant_Manufacturer'] = df.groupby('County')['Make'].transform(lambda x: x.value_counts().idxmax())
df['Dominant_Manufacturer_Proportion'] = df.groupby('County')['Make'].transform(lambda x: x.value_counts().max() / x.count())


In [32]:
print(df.columns)

Index(['VIN (1-10)', 'County', 'City', 'State', 'Postal Code', 'Model Year',
       'Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'DOL Vehicle ID', 'Vehicle Location', 'Electric Utility',
       '2020 Census Tract', 'Make_Proportion', 'Urban', 'CAFV Eligibility',
       'Urban_CAFV_Interaction_Urban_Not Eligible', 'Vehicle Age',
       'Yearly_Growth_Rate', 'Quarter', 'Month', 'BEV_Proportion',
       'PHEV_Proportion', 'Dominant_Manufacturer',
       'Dominant_Manufacturer_Proportion'],
      dtype='object')


In [33]:
df.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,...,CAFV Eligibility,Urban_CAFV_Interaction_Urban_Not Eligible,Vehicle Age,Yearly_Growth_Rate,Quarter,Month,BEV_Proportion,PHEV_Proportion,Dominant_Manufacturer,Dominant_Manufacturer_Proportion
0,1C4JJXP68P,Yakima,Yakima,WA,98901.0,2023,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,...,Not Eligible,False,1,0.454082,1,1,0.0,0.0,TESLA,0.432252
1,WBY8P6C05L,Kitsap,Kingston,WA,98346.0,2020,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,...,Not Eligible,False,4,-0.506024,1,1,0.0,0.0,TESLA,0.303797
2,JTDKARFP1J,Kitsap,Port Orchard,WA,98367.0,2018,TOYOTA,PRIUS PRIME,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,...,Not Eligible,False,6,0.149007,1,1,0.0,0.0,TESLA,0.303797
3,5UXTA6C09N,Snohomish,Everett,WA,98208.0,2022,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,...,Not Eligible,True,2,0.485420,1,1,0.0,0.0,TESLA,0.501246
4,JTMAB3FV7P,Thurston,Rainier,WA,98576.0,2023,TOYOTA,RAV4 PRIME,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,...,Not Eligible,True,1,1.195812,1,1,0.0,0.0,TESLA,0.310382


# Saving the Dataset

In [34]:
# Define the path where you want to save the dataset
output_path = '/content/gdrive/My Drive/Colab Notebooks/Electiric vehical/final_dataset_with_features.csv'

# Save the DataFrame to a CSV file
df.to_csv(output_path, index=False)

print(f"Dataset with engineered features saved to {output_path}")


Dataset with engineered features saved to /content/gdrive/My Drive/Colab Notebooks/Electiric vehical/final_dataset_with_features.csv
